# Final Project Workthrough

In [ ]:
# Code to set up the assignment
%cd /users/slin153/workspace/CMU10-714/Project-AMPA_2822B

In [ ]:
# Install Dependencies
%pip install -r ./requirements.txt

In [ ]:
!module load cuda
!make

In [ ]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

In [ ]:
import sys
sys.path.append('./python')

In [ ]:
# Download the datasets you will be using for this assignment

import urllib.request
import os

!mkdir -p './data/ptb'
# Download Penn Treebank dataset
ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
for f in ['train.txt', 'test.txt', 'valid.txt']:
    if not os.path.exists(os.path.join('./data/ptb', f)):
        urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))

# Download CIFAR-10 dataset
if not os.path.isdir("./data/cifar-10-batches-py"):
    urllib.request.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "./data/cifar-10-python.tar.gz")
    !tar -xvzf './data/cifar-10-python.tar.gz' -C './data'

-------------------------------------

## "ResNet9"

We now use your convolutional layer to implement a model similar to _ResNet9_, which is known to be a reasonable model for getting good accuracy on CIFAR-10 quickly (see [here](https://github.com/davidcpage/cifar10-fast)). Our main change is that we used striding instead of pooling and divided all of the channels by 4 for the sake of performance (as our framework is not as well-optimized as industry-grade frameworks).

<center><img src="https://github.com/dlsyscourse/hw4/blob/main/ResNet9.png?raw=true" alt="ResNet9" style="width: 400px;" /></center>


In [ ]:
!python3 -m pytest -l -v -k "resnet9"

Now we can train a ResNet on CIFAR10: 

In [ ]:
!python3 -m pytest -l -v -k "train_cifar10"

-----------------

Now, train our model on CIFAR-10 using the following code. Note that this is likely going to be quite slow, and also not all that accurate due to the lack of data augmentation. You should expect it to take around 500s per epoch.

In [ ]:
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
from models import ResNet9
from simple_ml import train_cifar10, evaluate_cifar10

device = ndl.cuda()
dataset = ndl.data.CIFAR10Dataset("data/cifar-10-batches-py", train=True)
dataloader = ndl.data.DataLoader(
      dataset=dataset,
      batch_size=128,
      shuffle=True,
)
model = ResNet9(device=device, dtype="float32")

start_time = time.time()
train_cifar10(
      model, 
      dataloader, 
      n_epochs=10, 
      optimizer=ndl.optim.Adam,
      lr=0.001, 
      weight_decay=0.001
)
train_time = time.time() - start_time
print(f"Training time: {train_time:.2f} seconds")

start_time = time.time()
evaluate_cifar10(model, dataloader)
eval_time = time.time() - start_time
print(f"Evaluation time: {eval_time:.2f} seconds")